# Introduction to Quantum Networking

Welcome to the fascinating world of **Quantum Networking**! 🌐⚛️

In this interactive notebook, you'll learn the fundamentals of quantum communication protocols and get hands-on experience with quantum key distribution, entanglement, and quantum network simulation.

## What is Quantum Networking?

Quantum networking leverages the principles of quantum mechanics to create ultra-secure communication channels. Unlike classical networks that transmit bits (0s and 1s), quantum networks transmit quantum bits (qubits) that can exist in superposition states.

### Key Concepts:
- **Quantum Key Distribution (QKD)**: Secure key sharing using quantum mechanics
- **Quantum Entanglement**: Spooky action at a distance for instant correlation
- **BB84 Protocol**: A pioneering quantum cryptography protocol
- **Quantum Channels**: The medium for transmitting qubits

### Learning Objectives:
By the end of this notebook, you will:
1. Understand quantum networking fundamentals
2. Implement quantum protocols from scratch
3. Interact with a live quantum network simulation
4. Write your own quantum host implementations
5. Analyze quantum security and error rates

Let's begin this quantum journey! 🚀


In [1]:
# Import required libraries for quantum networking
import numpy as np
import qutip as qt
import random
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Import our custom quantum networking modules
import sys
import os
sys.path.append('.')

print("🎯 Quantum Networking Environment Initialized!")
print("📚 Libraries loaded: QuTiP, NumPy, Matplotlib, IPyWidgets")
print("🔧 Ready for quantum experiments!")


🎯 Quantum Networking Environment Initialized!
📚 Libraries loaded: QuTiP, NumPy, Matplotlib, IPyWidgets
🔧 Ready for quantum experiments!


## Section 1: Quantum Fundamentals

Before diving into quantum networking, let's understand the basic building blocks:

### Qubits - The Quantum Bits

A qubit is the basic unit of quantum information. Unlike classical bits that are either 0 or 1, qubits can exist in a **superposition** of both states:

$$|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$$

Where $\alpha$ and $\beta$ are complex numbers called amplitudes, and $|\alpha|^2 + |\beta|^2 = 1$.


In [2]:
# Basic quantum state functions for networking
def create_qubit_states():
    """Create basic qubit states for quantum networking protocols"""
    # Basic computational basis states
    zero_state = qt.basis(2, 0)  # |0⟩
    one_state = qt.basis(2, 1)   # |1⟩
    
    # Superposition states (used in BB84 protocol)
    plus_state = (zero_state + one_state).unit()   # |+⟩ = (|0⟩ + |1⟩)/√2
    minus_state = (zero_state - one_state).unit()  # |-⟩ = (|0⟩ - |1⟩)/√2
    
    return {
        'Z_basis': {'0': zero_state, '1': one_state},
        'X_basis': {'+': plus_state, '-': minus_state}
    }

def prepare_qubit(basis, bit):
    """Prepare a qubit in the specified basis and bit value"""
    states = create_qubit_states()
    
    if basis == "Z":
        return states['Z_basis'][str(bit)]
    elif basis == "X":
        return states['X_basis']['+' if bit == 0 else '-']
    else:
        raise ValueError("Basis must be 'Z' or 'X'")

def measure_qubit(qubit, basis):
    """Measure a qubit in the specified basis"""
    if basis == "Z":
        # Computational basis measurement
        projector0 = qt.ket2dm(qt.basis(2, 0))
        projector1 = qt.ket2dm(qt.basis(2, 1))
    elif basis == "X":
        # Hadamard basis measurement
        plus_state = (qt.basis(2, 0) + qt.basis(2, 1)).unit()
        minus_state = (qt.basis(2, 0) - qt.basis(2, 1)).unit()
        projector0 = qt.ket2dm(plus_state)
        projector1 = qt.ket2dm(minus_state)
    else:
        raise ValueError("Basis must be 'Z' or 'X'")
    
    prob0 = qt.expect(projector0, qubit)
    return 0 if random.random() < prob0 else 1

print("🎯 Quantum State Functions Ready!")
print("📦 Available functions: create_qubit_states(), prepare_qubit(), measure_qubit()")


🎯 Quantum State Functions Ready!
📦 Available functions: create_qubit_states(), prepare_qubit(), measure_qubit()


## Section 2: Quantum Network Simulation Interface

Now let's integrate with the live quantum network simulation running on your local server. This interface allows you to interact with the quantum network simulation directly from this notebook.


In [3]:
# Embed the quantum network simulation UI directly in the notebook
from IPython.display import IFrame, display
import subprocess
import threading
import time
import os
import requests
import signal

# Global variables to track server processes
backend_process = None
frontend_process = None

def start_backend_server():
    """Start the Python backend server"""
    global backend_process
    print("🚀 Starting backend server...")
    
    try:
        # Start the backend server using python start.py
        backend_process = subprocess.Popen(
            ["python", "start.py"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            cwd=".",
            creationflags=subprocess.CREATE_NEW_PROCESS_GROUP if os.name == 'nt' else 0
        )
        
        # Give it time to start
        time.sleep(3)
        
        # Check if it's running
        if check_server_status("http://localhost:5174/api/simulation/status/"):
            print("✅ Backend server started successfully!")
            return True
        else:
            print("⚠️ Backend server may still be starting...")
            return True
            
    except Exception as e:
        print(f"❌ Failed to start backend server: {e}")
        return False

def start_frontend_server():
    """Start the React frontend server"""
    global frontend_process
    print("🚀 Starting frontend server...")
    
    try:
        # Change to ui directory and start npm dev server
        frontend_process = subprocess.Popen(
            ["npm", "run", "dev"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            cwd="ui",
            creationflags=subprocess.CREATE_NEW_PROCESS_GROUP if os.name == 'nt' else 0
        )
        
        # Give it time to start
        time.sleep(5)
        
        # Check if it's running
        if check_server_status("http://localhost:5173"):
            print("✅ Frontend server started successfully!")
            return True
        else:
            print("⚠️ Frontend server may still be starting...")
            return True
            
    except Exception as e:
        print(f"❌ Failed to start frontend server: {e}")
        return False

def check_server_status(url, timeout=3):
    """Check if a server is running at the given URL"""
    try:
        response = requests.get(url, timeout=timeout)
        return response.status_code in [200, 404]  # 404 is ok for some endpoints
    except requests.exceptions.RequestException:
        return False

def start_simulation_servers():
    """Start both backend and frontend servers"""
    print("🎯 Starting Quantum Network Simulation Servers")
    print("=" * 50)
    
    # Check if servers are already running
    backend_running = check_server_status("http://localhost:5174/api/simulation/status/")
    frontend_running = check_server_status("http://localhost:5173")
    
    if backend_running and frontend_running:
        print("✅ Both servers are already running!")
        return True
    
    success = True
    
    # Start backend if not running
    if not backend_running:
        success &= start_backend_server()
    else:
        print("✅ Backend server already running")
    
    # Start frontend if not running  
    if not frontend_running:
        success &= start_frontend_server()
    else:
        print("✅ Frontend server already running")
    
    if success:
        print("\n🎉 Simulation servers are ready!")
        print("🌐 Backend: http://localhost:5174")
        print("🖥️ Frontend: http://localhost:5173")
    
    return success

def stop_simulation_servers():
    """Stop both simulation servers"""
    global backend_process, frontend_process
    
    print("🛑 Stopping simulation servers...")
    
    if backend_process:
        try:
            if os.name == 'nt':  # Windows
                backend_process.send_signal(signal.CTRL_BREAK_EVENT)
            else:  # Unix/Linux
                backend_process.terminate()
            backend_process.wait(timeout=5)
            print("✅ Backend server stopped")
        except:
            if backend_process:
                backend_process.kill()
            print("⚠️ Backend server force killed")
        backend_process = None
    
    if frontend_process:
        try:
            if os.name == 'nt':  # Windows
                frontend_process.send_signal(signal.CTRL_BREAK_EVENT)
            else:  # Unix/Linux
                frontend_process.terminate()
            frontend_process.wait(timeout=5)
            print("✅ Frontend server stopped")
        except:
            if frontend_process:
                frontend_process.kill()
            print("⚠️ Frontend server force killed")
        frontend_process = None

def show_simulation_interface():
    """Display the quantum network simulation interface"""
    print("🌐 Loading Quantum Network Simulation Interface...")
    
    # First, try to start servers if they're not running
    if start_simulation_servers():
        # Wait a moment for servers to fully initialize
        time.sleep(2)
        
        # Embed the localhost UI in an iframe
        iframe = IFrame(src="http://localhost:5173", width=1200, height=800)
        display(iframe)
        
        print("\n📋 Interface Guide:")
        print("• Use the canvas to create quantum nodes and connections")
        print("• Start/stop simulations using the control panel")
        print("• Monitor quantum key distribution in real-time")
        print("• Observe entanglement and quantum channel behavior")
        print("\n💡 Tip: If the interface doesn't load, wait a moment and run this cell again")
    else:
        print("❌ Failed to start simulation servers")
        print("💡 Try running the cells above to start servers manually")

def check_simulation_server():
    """Check if the quantum simulation server is accessible"""
    backend_running = check_server_status("http://localhost:5174/api/simulation/status/")
    frontend_running = check_server_status("http://localhost:5173")
    
    if backend_running and frontend_running:
        print("✅ Both simulation servers are running!")
        print("🌐 Backend: http://localhost:5174")
        print("🖥️ Frontend: http://localhost:5173")
        return True
    elif backend_running:
        print("✅ Backend server is running")
        print("❌ Frontend server not accessible")
        return False
    elif frontend_running:
        print("❌ Backend server not accessible")
        print("✅ Frontend server is running")
        return False
    else:
        print("❌ Both servers are not accessible")
        print("💡 Run start_simulation_servers() to start them automatically")
        return False

# Check initial server status
print("🔍 Checking simulation server status...")
server_running = check_simulation_server()

if not server_running:
    print("\n🚀 To start the simulation, run: start_simulation_servers()")
    print("🌐 To show the interface, run: show_simulation_interface()")
    print("🛑 To stop servers when done, run: stop_simulation_servers()")


🔍 Checking simulation server status...
✅ Both simulation servers are running!
🌐 Backend: http://localhost:5174
🖥️ Frontend: http://localhost:5173


In [4]:
# 🚀 One-Click Simulation Startup
# Run this cell to automatically start servers and show the simulation interface

print("🎯 Starting Quantum Network Simulation...")
print("⏳ This may take a few moments to start all services...")

# Automatically start and show the simulation
show_simulation_interface()

print("\n💡 Helpful Commands:")
print("🔍 Check status: check_simulation_server()")
print("🚀 Start servers: start_simulation_servers()")
print("🌐 Show interface: show_simulation_interface()")
print("🛑 Stop servers: stop_simulation_servers()")


🎯 Starting Quantum Network Simulation...
⏳ This may take a few moments to start all services...
🌐 Loading Quantum Network Simulation Interface...
🎯 Starting Quantum Network Simulation Servers
✅ Both servers are already running!



📋 Interface Guide:
• Use the canvas to create quantum nodes and connections
• Start/stop simulations using the control panel
• Monitor quantum key distribution in real-time
• Observe entanglement and quantum channel behavior

💡 Tip: If the interface doesn't load, wait a moment and run this cell again

💡 Helpful Commands:
🔍 Check status: check_simulation_server()
🚀 Start servers: start_simulation_servers()
🌐 Show interface: show_simulation_interface()
🛑 Stop servers: stop_simulation_servers()


## Section 3: Interactive Quantum Host Programming

Now for the exciting part! Instead of using hardcoded quantum hosts, you'll implement your own quantum networking protocols. This is where you learn by coding!

### Your Mission: Implement a Student Quantum Host

You'll create a `StudentQuantumHost` class that can:
1. Send and receive qubits
2. Implement the BB84 quantum key distribution protocol
3. Handle quantum measurements and basis reconciliation
4. Manage quantum entanglement


In [8]:
# Student Quantum Host Template - Fill in the missing code!

class StudentQuantumHost:
    """
    Interactive Quantum Host for learning quantum networking protocols.
    Students will implement the missing methods to complete the functionality.
    """
    
    def __init__(self, name, address):
        self.name = name
        self.address = address
        self.qubits_received = []
        self.basis_choices = []
        self.measurement_outcomes = []
        self.shared_key = []
        
        print(f"🎯 Created StudentQuantumHost: {name} at address {address}")
    
    def send_qubits_bb84(self, num_qubits=10):
        """
        TODO: Implement BB84 qubit generation and sending
        
        Steps:
        1. Generate random bits and bases for each qubit
        2. Prepare qubits in the chosen basis
        3. Store your choices for later reconciliation
        4. Return the prepared qubits
        """
        print(f"📤 {self.name}: Generating {num_qubits} qubits for BB84...")
        
        # YOUR CODE HERE
        qubits_to_send = []
        self.basis_choices = []
        self.measurement_outcomes = []
        
        for i in range(num_qubits):
            # Step 1: Choose random bit (0 or 1) and basis ('Z' or 'X')
            bit = random.choice([0, 1])
            basis = random.choice(['Z', 'X'])
            
            # Step 2: Prepare the qubit using our prepare_qubit function
            qubit = prepare_qubit(basis, bit)
            
            # Step 3: Store your choices
            self.basis_choices.append(basis)
            self.measurement_outcomes.append(bit)
            qubits_to_send.append(qubit)
        
        print(f"✅ Generated {len(qubits_to_send)} qubits")
        return qubits_to_send
    
    def receive_and_measure_qubits(self, qubits):
        """
        TODO: Implement qubit reception and measurement
        
        Steps:
        1. For each received qubit, choose a random measurement basis
        2. Measure the qubit in that basis
        3. Store your basis choices and measurement results
        """
        print(f"📥 {self.name}: Receiving {len(qubits)} qubits...")
        
        # YOUR CODE HERE
        self.basis_choices = []
        self.measurement_outcomes = []
        
        for qubit in qubits:
            # Step 1: Choose random measurement basis
            basis = random.choice(['Z', 'X'])
            
            # Step 2: Measure the qubit
            outcome = measure_qubit(qubit, basis)
            
            # Step 3: Store results
            self.basis_choices.append(basis)
            self.measurement_outcomes.append(outcome)
        
        print(f"✅ Measured {len(qubits)} qubits")
        return self.basis_choices, self.measurement_outcomes
    
    def reconcile_bases(self, other_host_bases):
        """
        TODO: Implement basis reconciliation for BB84
        
        Compare your basis choices with the other host's bases.
        Keep only the measurements where both hosts used the same basis.
        """
        print(f"🔄 {self.name}: Reconciling bases...")
        
        # YOUR CODE HERE
        shared_indices = []
        shared_key_bits = []
        
        for i, (my_basis, their_basis) in enumerate(zip(self.basis_choices, other_host_bases)):
            if my_basis == their_basis:
                shared_indices.append(i)
                shared_key_bits.append(self.measurement_outcomes[i])
        
        self.shared_key = shared_key_bits
        
        print(f"✅ Found {len(shared_indices)} matching bases")
        print(f"🔑 Shared key bits: {self.shared_key}")
        return shared_indices, self.shared_key
    
    def estimate_error_rate(self, other_host, shared_indices, sample_fraction=0.3):
        """
        TODO: Implement error rate estimation
        
        Compare a sample of your shared key bits with the other host
        to estimate if there's an eavesdropper present.
        """
        print(f"🔍 {self.name}: Estimating error rate...")
        
        # YOUR CODE HERE
        if not shared_indices:
            return 0
        
        # Take a sample of the shared bits
        sample_size = max(1, int(len(shared_indices) * sample_fraction))
        sample_indices = random.sample(shared_indices, sample_size)
        
        errors = 0
        for idx in sample_indices:
            if self.measurement_outcomes[idx] != other_host.measurement_outcomes[idx]:
                errors += 1
        
        error_rate = errors / sample_size if sample_size > 0 else 0
        
        print(f"📊 Error rate: {error_rate:.2%} ({errors}/{sample_size} errors)")
        
        if error_rate > 0.1:  # 10% threshold
            print("⚠️  HIGH ERROR RATE - Potential eavesdropper detected!")
        else:
            print("✅ LOW ERROR RATE - Communication appears secure!")
        
        return error_rate

# Test the StudentQuantumHost
alice = StudentQuantumHost("Alice", "alice@quantum.net")
bob = StudentQuantumHost("Bob", "bob@quantum.net")

print("\n🎓 Student Quantum Hosts created! Ready for BB84 protocol implementation.")


🎯 Created StudentQuantumHost: Alice at address alice@quantum.net
🎯 Created StudentQuantumHost: Bob at address bob@quantum.net

🎓 Student Quantum Hosts created! Ready for BB84 protocol implementation.


In [9]:
# Interactive BB84 Protocol Demo
def run_bb84_demo():
    """Run a complete BB84 quantum key distribution demo"""
    print("🚀 Starting BB84 Quantum Key Distribution Demo")
    print("=" * 50)
    
    # Step 1: Alice generates and sends qubits
    print("\n🎯 Step 1: Alice generates qubits")
    qubits = alice.send_qubits_bb84(num_qubits=20)
    
    # Step 2: Bob receives and measures qubits
    print("\n🎯 Step 2: Bob measures received qubits")
    bob_bases, bob_outcomes = bob.receive_and_measure_qubits(qubits)
    
    # Step 3: Basis reconciliation
    print("\n🎯 Step 3: Public basis comparison")
    print(f"Alice's bases: {alice.basis_choices}")
    print(f"Bob's bases:   {bob.basis_choices}")
    
    shared_indices, shared_key = bob.reconcile_bases(alice.basis_choices)
    alice.reconcile_bases(bob.basis_choices)  # Alice does the same
    
    # Step 4: Error rate estimation
    print("\n🎯 Step 4: Error rate estimation")
    error_rate = bob.estimate_error_rate(alice, shared_indices)
    
    # Step 5: Final results
    print("\n🎯 Final Results:")
    print(f"✅ Alice's final key: {alice.shared_key}")
    print(f"✅ Bob's final key:   {bob.shared_key}")
    print(f"🔒 Keys match: {alice.shared_key == bob.shared_key}")
    
    return alice.shared_key, bob.shared_key, error_rate

# Widget for interactive demo
def create_interactive_demo():
    """Create interactive widgets for the BB84 demo"""
    
    # Widgets
    num_qubits_slider = widgets.IntSlider(
        value=15,
        min=5,
        max=50,
        step=5,
        description='Qubits:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    
    run_button = widgets.Button(
        description='Run BB84 Demo',
        disabled=False,
        button_style='success',
        tooltip='Click to run the quantum key distribution demo',
        icon='play'
    )
    
    output = widgets.Output()
    
    def on_button_click(b):
        with output:
            clear_output(wait=True)
            # Reset hosts
            alice.__init__("Alice", "alice@quantum.net")
            bob.__init__("Bob", "bob@quantum.net")
            
            # Run demo with selected number of qubits
            qubits = alice.send_qubits_bb84(num_qubits_slider.value)
            bob.receive_and_measure_qubits(qubits)
            shared_indices, _ = bob.reconcile_bases(alice.basis_choices)
            alice.reconcile_bases(bob.basis_choices)
            error_rate = bob.estimate_error_rate(alice, shared_indices)
            
            print(f"\n🎉 Demo completed with {num_qubits_slider.value} qubits!")
    
    run_button.on_click(on_button_click)
    
    return widgets.VBox([
        widgets.HBox([num_qubits_slider, run_button]),
        output
    ])

print("🎮 Interactive BB84 demo ready!")
print("📝 Run run_bb84_demo() or use the interactive widget below")


🎮 Interactive BB84 demo ready!
📝 Run run_bb84_demo() or use the interactive widget below


In [10]:
# Create and display the interactive demo widget
demo_widget = create_interactive_demo()
display(demo_widget)


## Section 4: Testing and Integration with Real Simulation

Now let's test our implementation and integrate it with the actual quantum network simulation.


In [11]:
# Test the basic quantum functions first
print("🧪 Testing Basic Quantum Functions")
print("=" * 40)

# Test qubit preparation
test_qubit_z0 = prepare_qubit("Z", 0)
test_qubit_z1 = prepare_qubit("Z", 1)
test_qubit_x0 = prepare_qubit("X", 0)
test_qubit_x1 = prepare_qubit("X", 1)

print("✅ Qubit preparation successful!")
print(f"   |0⟩ state: {test_qubit_z0}")
print(f"   |1⟩ state: {test_qubit_z1}")
print(f"   |+⟩ state: {test_qubit_x0}")
print(f"   |-⟩ state: {test_qubit_x1}")

# Test measurements
print("\n📊 Testing Measurements")
print("Measuring |0⟩ in Z basis:", measure_qubit(test_qubit_z0, "Z"))
print("Measuring |1⟩ in Z basis:", measure_qubit(test_qubit_z1, "Z"))
print("Measuring |+⟩ in X basis:", measure_qubit(test_qubit_x0, "X"))
print("Measuring |-⟩ in X basis:", measure_qubit(test_qubit_x1, "X"))

print("\n✅ Basic quantum functions working correctly!")


🧪 Testing Basic Quantum Functions
✅ Qubit preparation successful!
   |0⟩ state: Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]]
   |1⟩ state: Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [1.]]
   |+⟩ state: Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.70710678]]
   |-⟩ state: Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.70710678]
 [-0.70710678]]

📊 Testing Measurements
Measuring |0⟩ in Z basis: 0
Measuring |1⟩ in Z basis: 1
Measuring |+⟩ in X basis: 0
Measuring |-⟩ in X basis: 1

✅ Basic quantum functions working correctly!


In [12]:
# Test the complete BB84 protocol
print("🔐 Testing Complete BB84 Protocol")
print("=" * 40)

# Run a test with the student implementation
test_alice = StudentQuantumHost("Test_Alice", "test_alice@quantum.net")
test_bob = StudentQuantumHost("Test_Bob", "test_bob@quantum.net")

# Alice generates and sends qubits
test_qubits = test_alice.send_qubits_bb84(15)

# Bob receives and measures
test_bob.receive_and_measure_qubits(test_qubits)

# Basis reconciliation
shared_indices, _ = test_bob.reconcile_bases(test_alice.basis_choices)
test_alice.reconcile_bases(test_bob.basis_choices)

# Error rate estimation
error_rate = test_bob.estimate_error_rate(test_alice, shared_indices)

print(f"\n📈 Protocol Statistics:")
print(f"   Total qubits: {len(test_qubits)}")
print(f"   Matching bases: {len(shared_indices)}")
print(f"   Final key length: {len(test_bob.shared_key)}")
print(f"   Error rate: {error_rate:.2%}")
print(f"   Keys match: {test_alice.shared_key == test_bob.shared_key}")

print("\n✅ BB84 Protocol test completed successfully!")


🔐 Testing Complete BB84 Protocol
🎯 Created StudentQuantumHost: Test_Alice at address test_alice@quantum.net
🎯 Created StudentQuantumHost: Test_Bob at address test_bob@quantum.net
📤 Test_Alice: Generating 15 qubits for BB84...
✅ Generated 15 qubits
📥 Test_Bob: Receiving 15 qubits...
✅ Measured 15 qubits
🔄 Test_Bob: Reconciling bases...
✅ Found 7 matching bases
🔑 Shared key bits: [1, 0, 0, 1, 0, 0, 1]
🔄 Test_Alice: Reconciling bases...
✅ Found 7 matching bases
🔑 Shared key bits: [1, 0, 0, 1, 0, 0, 1]
🔍 Test_Bob: Estimating error rate...
📊 Error rate: 0.00% (0/2 errors)
✅ LOW ERROR RATE - Communication appears secure!

📈 Protocol Statistics:
   Total qubits: 15
   Matching bases: 7
   Final key length: 7
   Error rate: 0.00%
   Keys match: True

✅ BB84 Protocol test completed successfully!


In [13]:
# Integration with the actual simulation system
def integrate_with_simulation():
    """Show how to integrate the StudentQuantumHost with the real simulation"""
    print("🔗 Integration with Quantum Network Simulation")
    print("=" * 50)
    
    try:
        # Try to import the actual simulation components
        from quantum_network.interactive_host import InteractiveQuantumHost
        print("✅ Successfully imported InteractiveQuantumHost")
        
        # Create an interactive host that can use our student implementation
        class StudentImplementation:
            """Student's custom implementation for the InteractiveQuantumHost"""
            
            def prepare_qubit(self, basis, bit):
                return prepare_qubit(basis, bit)
            
            def measure_qubit(self, qubit, basis):
                return measure_qubit(qubit, basis)
            
            def bb84_send_qubits(self, num_qubits):
                print(f"🎓 Using student's BB84 implementation for {num_qubits} qubits")
                # Use the student's logic here
                return True
        
        # This is how students can integrate their code with the real simulation
        student_impl = StudentImplementation()
        
        print("🎯 To use your implementation in the real simulation:")
        print("1. Create an InteractiveQuantumHost")
        print("2. Pass your StudentImplementation to it")
        print("3. The simulation will use your custom quantum protocols!")
        
        print("\n📖 Example code:")
        print("```python")
        print("from quantum_network.interactive_host import InteractiveQuantumHost")
        print("my_host = InteractiveQuantumHost(..., student_implementation=student_impl)")
        print("```")
        
    except ImportError as e:
        print(f"⚠️  Could not import simulation components: {e}")
        print("💡 Make sure you're in the correct directory with the quantum_network module")
    
    print("\n✅ Integration explanation completed!")

# Run the integration explanation
integrate_with_simulation()


🔗 Integration with Quantum Network Simulation
⚠️  Could not import simulation components: No module named 'pydantic_settings'
💡 Make sure you're in the correct directory with the quantum_network module

✅ Integration explanation completed!


## 🎓 Congratulations! You've Completed the Quantum Networking Tutorial

### What You've Learned:
1. ✅ **Quantum Fundamentals**: Qubits, superposition, and measurement
2. ✅ **BB84 Protocol**: Quantum key distribution implementation
3. ✅ **Interactive Programming**: Building quantum hosts from scratch
4. ✅ **Simulation Integration**: Connecting with live quantum network simulation
5. ✅ **Security Analysis**: Error rate estimation and eavesdropper detection

### Next Steps:
- 🔬 **Experiment**: Try different parameters in the BB84 protocol
- 🎮 **Explore**: Use the simulation interface to create complex networks
- 🛡️ **Security**: Implement eavesdropper detection algorithms
- 🌐 **Extend**: Add more quantum protocols like E91 or SARG04
- 🤝 **Collaborate**: Share your implementations with other students

### Key Resources:
- **Interactive Simulation**: Use `show_simulation_interface()` to access the web UI
- **Your Implementation**: StudentQuantumHost class ready for experimentation
- **Real Integration**: quantum_network/interactive_host.py for production use

**Happy Quantum Networking!** 🚀⚛️
